In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm
import numpy as np
from dask.distributed import Client, progress, LocalCluster
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar
plt.rcParams['figure.dpi']=400

In [ ]:
def denseH_MBL(L, J, hx, hz, periodic):
    """
    Generates the dense Hamiltonian matrix for the MBL model with quenched disorder.
    
    Parameters:
        L (int): length of chain
        J (float): Ising interaction strength
        hx (ndarray): array of transverse field strengths
        hz (ndarray): array of longitudinal field strengths
        periodic (bool): does the chain have periodic boundary conditions?
        
    Returns:
        H (ndarray): 2^L x 2^L matrix representing the Hamiltonian operator
    """
    
    dim = 2 ** L  # dimensions of the Hilbert space
    H = np.zeros((dim, dim))  # initialize the Hamiltonian
    
    # Calculation of off-diagonal elements due to the magnetic field
    for beta in range(dim):  # iterate over all states
        for j in range(1, L + 1):  # iterate over all sites
            alpha = beta ^ (1 << (j - 1))  # flips jth bit of beta to get the state alpha
            H[alpha, beta] -= hx[j - 1]  # contribution by sigma^x_j
    
    # Calculation of diagonal elements due to Ising interaction and longitudinal magnetic field
    for alpha in range(dim):  # iterate over all states
        for j in range(1, L):  # iterate over all sites
            if 2 * (alpha & (1 << (j - 1))) == (alpha & (1 << j)):  # same spin
                H[alpha, alpha] -= J  # decrease the energy by the Ising interaction term
            else:  # different spin
                H[alpha, alpha] += J  # increase the energy by the Ising interaction term
            
            # Longitudinal magnetic field contribution
            if alpha & (1 << (j - 1)) == 0:  # spin up
                H[alpha, alpha] -= hz[j - 1]  # decrease energy for spin up
            else:  # spin down
                H[alpha, alpha] += hz[j - 1]  # increase energy for spin down
        
        # Handling periodic boundary conditions
        if periodic and L > 1:  # L > 1 needed for periodicity to mean anything
            if (alpha & (1 << (L - 1))) == ((alpha & (1 << 0)) * (2 ** (L - 1))):  # same spin at ends
                H[alpha, alpha] -= J  # decrease the energy by the Ising interaction term
            else:  # different spin at ends
                H[alpha, alpha] += J  # increase the energy by the Ising interaction term
    
    return H

def diagonalize_MBL(Ls, J, hx_list, hz_list, periodic):
    eigss = []
    vecss = []
    
    for i, L in enumerate(Ls):
        hx = hx_list[i]
        hz = hz_list[i]
        H = denseH_MBL(L, J, hx, hz, periodic)
        eigs, vecs = scipy.linalg.eigh(H)
        eigss.append(eigs)
        vecss.append(vecs)
    
    return eigss, vecss

# Parameters
Ls = [6, 8, 10, 12, 14]
J = 1
W = 3
periodic = True

# Generate random fields hx and hz
hx_list = [np.random.uniform(-W, W, L) for L in Ls]
hz_list = [np.random.uniform(-W, W, L) for L in Ls]

# Diagonalize
start_time = time.time()
eigss, vecss = diagonalize_MBL(Ls, J, hx_list, hz_list, periodic)
end_time = time.time()
print('time taken:', end_time-start_time)
